# **Prequisite**

In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models

# **Data Preprocessing**

In [10]:
folder_path = "dataset/dataset"
os.listdir(folder_path)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [11]:
classes=["cardboard","glass","metal","paper","plastic","trash"]

In [12]:
data=[]
for c in classes:
    i=classes.index(c)
    path=os.path.join(folder_path,c)
    for img in os.listdir(path):
        im=cv2.imread(os.path.join(path,img))
        im=cv2.resize(im,(224,224))
        data.append([im,i])

In [13]:
data=np.array(data , dtype="object")
import random
random.shuffle(data)

In [14]:
X=[]
Y=[]
for x,y in data:
    X.append(x)
    Y.append(y)

In [16]:
x=np.array(X)
y=np.array(Y)
print(x.shape,y.shape)

(2527, 224, 224, 3) (2527,)


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=4, stratify=y)

In [18]:
x_train = x_train / 255.0
x_test = x_test / 255.0

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(1516, 224, 224, 3) (1516,)
(1011, 224, 224, 3) (1011,)
